In [1]:
import pandas as pd
import numpy as np
import glob
import datetime as dt

In [2]:
file_paths = glob.glob("/content/drive/MyDrive/NFL Analytics Project/data/final_df_nfl_*.csv")
dataframes = []

for file_path in file_paths:
    df = pd.read_csv(file_path)
    dataframes.append(df)

final_df = pd.concat(dataframes, ignore_index=True)
final_df = final_df.sort_values(by='Date', ascending=True)
final_df = final_df.drop(columns = {'Unnamed: 0'})
final_df = final_df[final_df['Total'] != 0]
final_df['Date'] = pd.to_datetime(final_df['Date'])
final_df['Year'] = final_df['Date'].dt.year
top_5_rows = final_df.head(5)
markdown_table = top_5_rows.to_markdown(index=False)
print(markdown_table)

|   Week | Date                | Matchup          | Final Score   |   Total Points | Winner   | Home_Away_Winner   | Away Team   |   Away Score |   Away Score Differential | Home Team   |   Home Score |   Home Score Differential |   Away Spread |   Home Spread |   Away Consensus |   Home Consensus | Con. Spread Favorite   | Con. Spread Underdog   |   Total |   Over Consensus |   Under Consensus | Con. Total Favorite   | Con. Total Underdog   |   Win Marginal | Spread Covering Team   | Totals Covering Line   | Consensus Spread Winner   | Total Consensus Winner   |   Year |
|-------:|:--------------------|:-----------------|:--------------|---------------:|:---------|:-------------------|:------------|-------------:|--------------------------:|:------------|-------------:|--------------------------:|--------------:|--------------:|-----------------:|-----------------:|:-----------------------|:-----------------------|--------:|-----------------:|------------------:|:---------------------

In [ ]:
final_df.columns

Index(['Week', 'Date', 'Matchup', 'Final Score', 'Total Points', 'Winner',
       'Home_Away_Winner', 'Away Team', 'Away Score',
       'Away Score Differential', 'Home Team', 'Home Score',
       'Home Score Differential', 'Away Spread', 'Home Spread',
       'Away Consensus', 'Home Consensus', 'Con. Spread Favorite',
       'Con. Spread Underdog', 'Total', 'Over Consensus', 'Under Consensus',
       'Con. Total Favorite', 'Con. Total Underdog', 'Win Marginal',
       'Spread Covering Team', 'Totals Covering Line',
       'Consensus Spread Winner', 'Total Consensus Winner', 'Year'],
      dtype='object')

In [ ]:
final_df['Con. Total Favorite'].value_counts()

Over     1265
Under     587
Name: Con. Total Favorite, dtype: int64

#Questions
- 1)Who wins more games Home vs. Away team?
- 2)Does the Over/Under hit more often?
- 3)Does + Spread or - Spread hit more often?
- 4)Does the Consensus favorite or underdog win more often? (Spread)
- 5)Does the Consensus favorite or underdog win more often? (Totals)
- 6)Is it more common to bet over vs under? (Consensus)
- 7)Is it more common to bet - vs +? (Consensus)



## 1)Who wins more games Home vs. Away team?

In [3]:
print(round(final_df['Home_Away_Winner'].value_counts(normalize=True),4))
winner = round((final_df['Home_Away_Winner'].value_counts(normalize=True).max() * 100),2)
print('')
print(f'The Home team wins {winner}% of games')

Home    0.5513
Away    0.4487
Name: Home_Away_Winner, dtype: float64

The Home team wins 55.13% of games


#2)Does the Over/Under hit more often?


In [4]:
print(round(final_df['Totals Covering Line'].value_counts(normalize=True),4))
winner = round((final_df['Totals Covering Line'].value_counts(normalize=True).max() * 100),2)
print('')
print(f'The Under hits {winner}% of games')

Under    0.5146
Over     0.4719
Push     0.0135
Name: Totals Covering Line, dtype: float64

The Under hits 51.46% of games


# 3)Does + Spread or - Spread hit more often?


In [5]:
# Assuming you have the DataFrame named df
final_df['Spread Winner'] = ''

for index, row in final_df.iterrows():
    spread_covering_team = row['Spread Covering Team']
    away_team = row['Away Team']
    away_spread = row['Away Spread']
    home_team = row['Home Team']
    home_spread = row['Home Spread']

    if spread_covering_team == away_team and str(away_spread).startswith('-'):
        result = '-'
    elif spread_covering_team == away_team  and '-' not in str(away_spread):
        result = '+'
    elif spread_covering_team == home_team and str(home_spread).startswith('-'):
        result = '-'
    elif spread_covering_team == home_team and '-' not in str(home_spread):
        result = '+'
    else:
        result = 'N/A'

    final_df.at[index, 'Spread Winner'] = result

# Print the updated DataFrame
final_df[['Spread Covering Team',
          'Away Team',
          'Away Spread',
          'Home Team',
          'Home Spread',
          'Spread Winner']].tail(5)

,Spread Covering Team,Away Team,Away Spread,Home Team,Home Spread,Spread Winner
747,49ers,Cardinals,14.0,49ers,-14.0,-
748,Rams,Rams,4.5,Seahawks,-4.5,+
749,Commanders,Cowboys,-7.5,Commanders,7.5,+
750,Lions,Lions,3.5,Packers,-3.5,+
741,Texans,Texans,3.0,Colts,-3.0,+


In [6]:
print(round(final_df['Spread Winner'].value_counts(normalize=True),4))
winner = round((final_df['Spread Winner'].value_counts(normalize=True).max() * 100),2)
print('')
print(f'The Spread Underdog covers {winner}% of games')

+    0.5076
-    0.4924
Name: Spread Winner, dtype: float64

The Spread Underdog covers 50.76% of games


#4)Does the Consensus favorite or underdog win more often? (Spread)

In [7]:
print(round(final_df['Consensus Spread Winner'].value_counts(normalize=True),4))
winner = round((final_df['Consensus Spread Winner'].value_counts(normalize=True).max() * 100),2)
print('')
print(f'The Spread Consensus Majority Spread covers {winner}% of games')

Majority    0.4860
Minority    0.4838
Push        0.0302
Name: Consensus Spread Winner, dtype: float64

The Spread Consensus Majority Spread covers 48.6% of games


# 5)Does the Consensus favorite or underdog win more often? (Totals)

In [8]:
print(round(final_df['Total Consensus Winner'].value_counts(normalize=True),4))
winner = round((final_df['Total Consensus Winner'].value_counts(normalize=True).max() * 100),2)
print('')
print(f'The Totals Consensus Minority Spread covers {winner}% of games')

Minority    0.5063
Majority    0.4533
Push        0.0405
Name: Total Consensus Winner, dtype: float64

The Totals Consensus Minority Spread covers 50.63% of games


# 6) Is it more common to bet over vs under? (Consensus)


In [9]:
print(round(final_df['Con. Total Favorite'].value_counts(normalize=True),4))
winner = round((final_df['Con. Total Favorite'].value_counts(normalize=True).max() * 100),2)
print('')
print(f'The General Public prefers to bet on the Over {winner}% of games')

Over     0.683
Under    0.317
Name: Con. Total Favorite, dtype: float64

The General Public prefers to bet on the Over 68.3% of games


# 7)Is it more common to bet - vs +? (Consensus)

In [11]:
# Create the new columns
# Create the new columns

# import pandas as pd

# data = {
#     'Away Spread': [-3.0, 3, -5, 6, 3],
#     'Away Consensus': [.53, .57, .4, .34, .5],
#     'Home Spread': [3.0, -3, 5, -6, -3],
#     'Home Consensus': [.47, .43, .6, .66, .5]
# }

# final_df = pd.DataFrame(data)

# Create the 'Con. Line Favorite' column
final_df['Con. Line Favorite'] = ''

# Create the 'Con. Line Underdog' column
final_df['Con. Line Underdog'] = ''

# Iterate over the rows of the dataframe
for index, row in final_df.iterrows():
    away_spread = row['Away Spread']
    away_consensus = row['Away Consensus']
    home_spread = row['Home Spread']
    home_consensus = row['Home Consensus']

    if home_consensus == 0.5:
        final_df.at[index, 'Con. Line Favorite'] = 'Equal'
        final_df.at[index, 'Con. Line Underdog'] = 'Equal'
    elif away_spread < 0 and away_consensus > home_consensus:
        final_df.at[index, 'Con. Line Favorite'] = '-'
        final_df.at[index, 'Con. Line Underdog'] = '+'
    elif away_spread < 0 and away_consensus < home_consensus:
        final_df.at[index, 'Con. Line Favorite'] = '+'
        final_df.at[index, 'Con. Line Underdog'] = '-'
    elif home_spread < 0 and home_consensus > away_consensus:
        final_df.at[index, 'Con. Line Favorite'] = '-'
        final_df.at[index, 'Con. Line Underdog'] = '+'
    elif home_spread < 0 and home_consensus < away_consensus:
        final_df.at[index, 'Con. Line Favorite'] = '+'
        final_df.at[index, 'Con. Line Underdog'] = '-'

final_df[['Home Spread', 'Home Consensus', 'Away Spread', 'Away Consensus', 'Con. Line Favorite', 'Con. Line Underdog']].tail(20)

,Home Spread,Home Consensus,Away Spread,Away Consensus,Con. Line Favorite,Con. Line Underdog
731,-3.0,0.56,3.0,0.44,-,+
732,-6.5,0.48,6.5,0.52,+,-
726,-3.5,0.42,3.5,0.58,+,-
734,2.5,0.59,-2.5,0.41,+,-
736,-6.0,0.55,6.0,0.45,-,+
735,8.5,0.63,-8.5,0.37,+,-
737,-5.5,0.39,5.5,0.61,+,-
738,-8.5,0.55,8.5,0.45,-,+
739,6.5,0.39,-6.5,0.61,-,+
740,-12.5,0.55,12.5,0.45,-,+


In [13]:
print(round(final_df['Con. Line Favorite'].value_counts(normalize=True),4))
winner = round((final_df['Con. Line Favorite'].value_counts(normalize=True).max() * 100),2)
print('')
print(f'The General Public prefers to bet on the team Favorite to cover the spread in {winner}% of games')

-        0.6631
+        0.2910
Equal    0.0302
         0.0157
Name: Con. Line Favorite, dtype: float64

The General Public prefers to bet on the team Favorite to cover the spread in 66.31% of games


In [14]:
final_df.columns

Index(['Week', 'Date', 'Matchup', 'Final Score', 'Total Points', 'Winner',
       'Home_Away_Winner', 'Away Team', 'Away Score',
       'Away Score Differential', 'Home Team', 'Home Score',
       'Home Score Differential', 'Away Spread', 'Home Spread',
       'Away Consensus', 'Home Consensus', 'Con. Spread Favorite',
       'Con. Spread Underdog', 'Total', 'Over Consensus', 'Under Consensus',
       'Con. Total Favorite', 'Con. Total Underdog', 'Win Marginal',
       'Spread Covering Team', 'Totals Covering Line',
       'Consensus Spread Winner', 'Total Consensus Winner', 'Year',
       'Spread Winner', 'Con. Line Favorite', 'Con. Line Underdog'],
      dtype='object')

In [21]:
#import drive to set up Colab and Drive connection to access files in the Drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
path = "/content/drive/MyDrive/NFL Analytics Project/data/final_dataset.csv" #save the final dataset to drive

with open(path, 'w', encoding = 'utf-8-sig') as df:
  final_df.to_csv(df)
print('The final_df has been saved to the path: {}'.format('/content/drive/MyDrive/NFL Analytics Project/data/'))

The final_df has been saved to the path: /content/drive/MyDrive/NFL Analytics Project/data/
